In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

import numpy as np

In [35]:
NUM_TRAIN = 40000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=40000, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=10000)


Files already downloaded and verified
Files already downloaded and verified


In [36]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

for t, (x, y) in enumerate(loader_train):
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            
for t, (x, y) in enumerate(loader_test):
            x_Test = x.to(device=device, dtype=dtype)
            y_Test = y.to(device=device, dtype=torch.long)

using device: cuda


In [37]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)


In [38]:
channel_1 = 32
channel_2 = 64
channel_3 = 32


learning_rate = 2.5e-3
model = None
optimizer = None

in_channel = 3
num_classes = 10

model = nn.Sequential(
    nn.Conv2d(in_channel,channel_1, kernel_size=5, padding=2, stride=1),
    nn.ReLU(),
    nn.BatchNorm2d(num_features= channel_1),
    nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1, stride=1),
    nn.ReLU(),
    nn.BatchNorm2d(num_features= channel_2),
    nn.MaxPool2d(kernel_size=1,stride=1),
    nn.Conv2d(channel_2, channel_3, kernel_size=3, padding=1, stride=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_3 *32*32, num_classes)
)
model = model.to(device=device)

from skorch import NeuralNetClassifier

net = NeuralNetClassifier(
    module=model,
    criterion = nn.CrossEntropyLoss,
    optimizer=optim.Adam,
#     optimizer_momentum = 0.9,
    train_split=None,
    max_epochs=5,
    lr= learning_rate,
    warm_start = True,
    device = device

)

from sklearn.model_selection import KFold
from sklearn import metrics

# for epoch in range(5):
kf = KFold(n_splits=5, shuffle = True)
accuracies=[]
for train_index, test_index in kf.split(x):
    accuracies=[]
    xk_train, xk_test = x[train_index], x[test_index]
    yk_train, yk_test = y[train_index], y[test_index]
    net.fit(xk_train,yk_train)
    y_pred = net.predict(xk_test)
    acc = metrics.accuracy_score(yk_test, y_pred)
    accuracies.append(acc)
    print('FinalAccuracy %.4f' % (np.mean(accuracies)))
    
y_pred_test = net.predict(x_Test)
acc = metrics.accuracy_score(y_Test, y_pred_test)
print('TestAccuracy %.4f' % (acc))
    

  epoch    train_loss     dur
-------  ------------  ------
      1        2.3457  3.3018
      2        1.4419  3.3325
      3        1.2362  3.3169
      4        1.0985  3.2829
      5        0.9055  3.2535
FinalAccuracy 0.4560
      6        0.9641  3.3189
      7        0.6223  3.4624
      8        0.4775  3.3138
      9        0.4421  3.2777
     10        0.3262  3.2562
FinalAccuracy 0.5295
     11        0.5655  3.3662
     12        0.1910  3.4717
     13        0.1018  3.3429
     14        0.0734  3.3759
     15        0.0561  3.2931
FinalAccuracy 0.8015
     16        0.1825  3.3058
     17        0.0727  3.3169
     18        0.0407  3.3337
     19        0.0244  3.3259
     20        0.0114  3.3167
FinalAccuracy 0.9585
     21        0.0413  3.3199
     22        0.0256  3.3135
     23        0.0130  3.3239
     24        0.0044  3.3065
     25        0.0011  3.3747
FinalAccuracy 0.9920
TestAccuracy 0.9984
